<a href="https://colab.research.google.com/github/abyaadrafid/APTOS-Blindness-Detection/blob/master/autoencoder_self_supervision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geopandas

     |████████████████████████████████| 994 kB 17.4 MB/s 
     |████████████████████████████████| 15.4 MB 34 kB/s 
     |████████████████████████████████| 6.3 MB 39.0 MB/s 


In [2]:
import torch
import geopandas as gpd
import matplotlib.pyplot as plt
import cv2
from PIL import Image, ImageDraw
from osgeo import gdal
import numpy as np
import math
from tqdm import notebook
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, utils
import torchvision.models as models
import torch.nn as nn
from torch.optim import Adam

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
rootdir = "/content/drive/My Drive/tiles_updated/tiles/"

In [5]:
samples = os.listdir(rootdir)

In [27]:
def cooord2pixel(xp, yp, geo_transform):
  xoff, a, b, yoff, d, e  = geo_transform 
  y = (a*yp - d*xp + d*xoff - a*yoff) / (a*e - d*b)
  x = (xp - xoff - b * y)/a
  return x, y

In [68]:
def get_locations(sample):

  image_location = rootdir+sample+'/'+sample+'_RGBI.tif'
  annotation_location = rootdir+sample+'/'+ 'results' + sample[12:]+'/tree_crown_poly_smooth.shp'

  return image_location, annotation_location

In [80]:
def preprocess_dataset(samples) :
  
  data_points = []

  for i,_ in notebook.tqdm(enumerate(samples)) :
    
    image_location, annotation_location = get_locations(samples[i])
    gdf = gpd.GeoDataFrame.from_file(annotation_location)
    ds = gdal.Open(image_location).GetGeoTransform()

    for item in range(len(gdf)):

      x_coords, y_coords = gdf.geometry[item].exterior.xy
      x_coords = np.array(x_coords)
      y_coords = np.array(y_coords)

      x_points, y_points = cooord2pixel(x_coords, y_coords, ds)
      data_points.append((x_points,y_points,samples[i]))

  return data_points

In [81]:
data_list = preprocess_dataset(samples)

0it [00:00, ?it/s]

In [83]:
class PinaDataset(Dataset):

  def __init__(self, data_list):
    self.data_list = data_list
  
  def __len__(self):
    return len(self.data_list)

  def __getitem__(self, idx):

    

    return data_list[idx]

In [84]:
pdataset = PinaDataset(data_list)

In [85]:
pdataset[2]

(array([181.7 , 178.65, 173.6 , 171.9 , 172.  , 179.4 , 203.6 , 206.95,
        207.4 , 206.95, 205.15, 181.7 ]),
 array([35.45      , 33.15      , 28.10000001, 24.95      , 23.        ,
         2.35      ,  2.15      , 27.55      , 32.4       , 33.45      ,
        33.9       , 35.45      ]),
 'schlegel2019_683454_683214_5589229_5588989')